In [232]:
#%load_ext autoreload
#%autoreload 2
%matplotlib inline

import os
import sys
import imp

import numpy as np
import pandas as pd
import xray
from numba.decorators import jit, autojit

import climatools.aerosol.f2py3_modal_aero_wateruptake as aerowateruptake
import climatools.aerosol.aerosol as climaaerosol
import climatools.io.cesm as climaio
import climatools.rrtmg as climarrtmg

imp.reload(climaaerosol)

<module 'climatools.aerosol.aerosol' from '/nuwa_cluster/home/jackyu/climatools/climatools/aerosol/aerosol.py'>

In [78]:
# load a CAM history file from some previous global run
filedir = '/nuwa_cluster/home/jackyu/climatools/climatools/aerosol'
filepath = os.path.join(filedir, 'f19_g16.B1850.std.cam2.h0.0001-06.nc')
ds = climaio.load_camhistory(readfrom=filepath)

In [77]:
ILON, ILAT = 70, 90
ITIME = 0

In [55]:
LON, LAT = ds['lon'][ILON].values, ds['lat'][ILAT].values
TIME = ds['time'][ITIME].values

### timeit: aerowateruptake.modal_aer_wateruptake_sub()

In [208]:
pcols, pver, ntot_amode, max_nspec_amode = 1, 30, 3, 6
args_isel = {'time':ITIME, 'lon':ILON, 'lat':ILAT}
modes = [1, 2, 3]

try:
    relative_humidity = ds['RELHUM'].isel(**args_isel)
    cloud_fraction = ds['CLOUD'].isel(**args_isel)
    species_mmr = climaaerosol.get_raer(ds, **args_isel)
    
    relative_humidity = relative_humidity.values.reshape((pcols, pver))
    cloud_fraction = cloud_fraction.values.reshape((pcols, pver))
except KeyError as e:
    print('Needed variables missing from CAM history.')
    raise e

In [210]:
%timeit aerowateruptake.modal_aero_wateruptake_sub(ncol=1,\
            cldn=cloud_fraction,\
            relative_humidity=relative_humidity,\
            raer=species_mmr)

10000 loops, best of 3: 101 µs per loop


### timeit: get_aer()

In [264]:
%timeit climaaerosol.get_raer(ds, lon=ILON, lat=ILAT)

100 loops, best of 3: 7.22 ms per loop


### timeit: climaaerosol.wateruptake_column()

In [265]:
%timeit climaaerosol.wateruptake_column(ds, ilon=ILON, ilat=ILAT)

100 loops, best of 3: 9.21 ms per loop


### timeit: climaaerosol.wateruptake_column() inside loops

In [266]:

def runthis():
    for _ in range(96):
        for _ in range(180):
            climaaerosol.wateruptake_column(ds, ilon=ILON, ilat=ILAT)    
            
%timeit runthis()

1 loops, best of 3: 2min 38s per loop


In [240]:
144 * 96 * 10e-3 / (60)

2.3040000000000003

In [257]:
#%timeit climaaerosol.wateruptake(ds)